<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: mhdflux.C

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain how the MHD fluxes are evaluated within IllinoisGRMHD.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#introduction): **Introduction**
1. [Step 2](#basic_quantities): **Basic quantities**
1. [Step 3](#eos_quantities): **Equation of State (EOS) quantities**
1. [Step 4](#impose_speed_limit_output_u0): **Impose a speed limit and determine $u^{0}$**
1. [Step 5](#magnetic_field_comoving_frame): **The magnetic field measured in the comoving fluid frame, $b^{\mu}$**
1. [Step 6](#min_max_characteristic_speeds): **The minimum and maximum characteristic speeds**
1. [Step 7](#rho_star_flux): **MHD flux: $\rho_{\star}$**
1. [Step 8](#energy_flux): **MHD flux: $\tilde{\tau}$**
1. [Step 9](#momentum_flux): **MHD flux: $\tilde{S}_{i}$**
1. [Step 10](#code_validation): **Code validation**
1. [Step 11](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0b: Load up cmdline_helper and create the directory
import cmdline_helper as cmd
IGM_src_dir_path = os.path.join("..","src")
cmd.mkdir(IGM_src_dir_path)

# Step 0c: Create the output file path 
outfile_path__mhdflux__C = os.path.join(IGM_src_dir_path,"mhdflux.C")

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial module we will compute the flux terms for the conservative variables $U=\left\{\rho_{\star},\tilde{\tau},\tilde{S}_{i}\right\}$, which are defined in terms of the primitive variables as

$$
\left(
\begin{matrix}
\rho_{\star}\\
\tilde{\tau}\\
\tilde{S}_{i}
\end{matrix}
\right)
=
\left(
\begin{matrix}
\alpha\sqrt{\gamma}\rho_{b}u^{0}\\
\alpha^{2}\sqrt{\gamma}T^{00}-\rho_{\star}\\
\left(\rho_{\star}h + \alpha u^{0}\sqrt{\gamma}b^{2}\right)u_{i} - \alpha\sqrt{\gamma}b^{0}b_{i}
\end{matrix}
\right)\ .
$$

The flux terms for these conservative variables are

$$
\boldsymbol{F} 
= 
\left(
\begin{matrix}
F^{i}_{\rho_{\star}}\\
F^{i}_{\tilde{\tau}}\\
\left(F_{\tilde{S}}\right)^{j}_{\ i}
\end{matrix}
\right)
=
\left(
\begin{matrix}
\rho_{\star}v^{i}\\
\alpha^{2}\sqrt{\gamma}T^{0j} - \rho_{\star}v^{j}\\
\alpha\sqrt{\gamma}T^{j}_{\ i}
\end{matrix}
\right)\ .
$$

The MHD flux algorithm computes, for each of the fluxes above, the standard Harten-Lax-van Leer (HLL) flux

$$
\boxed{F^{\rm HLL} = \frac{c^{-}F_{r} + c^{+}F_{l} - c^{+}c^{-}\left(U_{r} - U_{l}\right)}{c^{+} + c^{-}}}\ .
$$

As we go through the implementation, we will notice that will need a lot of the functions defined within the `inlined_functions.C` code file of `IllinoisGRMHD`. We will therefore explain the algorithm of each function in quite some detail, so that the reader can go through this tutorial without having to stop and read the [`inlined_functions.C` tutorial module](Tutorial-IllinoisGRMHD_inlined_functions.ipynb). We do encourage the reader to go through the module as well, though, since we will be presenting the math behind each function, but not the code itself.

<a id='basic_quantities'></a>

# Step 2: Basic quantities \[Back to [top](#toc)\]
$$\label{basic_quantities}$$

We start by defining useful quantities, such as $\psi^{\pm4}$, $\psi^{6}$, $\alpha\sqrt{\gamma}$, $\alpha^{-1}$, and $\alpha^{-2}$.

In [2]:
%%writefile $outfile_path__mhdflux__C
//-----------------------------------------------------------------------------
// Compute the flux for advecting rho_star, tau (Font's energy variable), 
//  and S_i .
//-----------------------------------------------------------------------------
static inline void mhdflux(int i,int j,int k,const int flux_dirn,CCTK_REAL *Ul,CCTK_REAL *Ur,  CCTK_REAL *FACEVAL,eos_struct &eos,
                           CCTK_REAL &cmax,CCTK_REAL &cmin,
                           CCTK_REAL &rho_star_flux,CCTK_REAL &tau_flux,CCTK_REAL &st_x_flux,CCTK_REAL &st_y_flux,CCTK_REAL &st_z_flux) {
    
  DECLARE_CCTK_PARAMETERS;

  CCTK_REAL psi2  = FACEVAL[CM_PSI] * FACEVAL[CM_PSI];
  CCTK_REAL psi4  = psi2 * psi2;
  CCTK_REAL psi6  = psi2 * psi4;
  CCTK_REAL lapse = FACEVAL[CM_LAPM1] + 1.0;

  CCTK_REAL ONE_OVER_LAPSE = 1.0/lapse;

Writing ../src/mhdflux.C


<a id='eos_quantities'></a>

# Step 3: Equation of State (EOS) quantities \[Back to [top](#toc)\]
$$\label{eos_quantities}$$

Next we compute the quantities $P_{\rm cold}$, $\epsilon_{\rm cold}$, $\frac{dP_{\rm cold}}{d\rho}$, $\epsilon_{\rm th}$, $h$, and $\Gamma_{\rm cold}$. We have written $\rho_{b}\equiv\rho$ so that the discussion constains a notation that is slightly less cluttered with indices.

The function `compute_P_cold__eps_cold__dPcold_drho__eps_th__h__Gamma_cold()` is defined within the `inlined_functions.C` code file of `IllinoisGRMHD`. It currently implementes 3 different cases:

1. **Case 1: $\boldsymbol{\rho_{b} = 0}$**

   In this case, we set $\Gamma_{\rm cold} = \Gamma_{\rm tab}$, i.e. it's tabulated input value, and all other quantities to zero: $P_{\rm cold}=\epsilon_{\rm cold}=\frac{dP_{\rm cold}}{d\rho}=h=\epsilon_{\rm th}=0$
   
2. **Case 2: Single Polytrope EOS**

   In this case we have have the relations:
   
   $$
   \boxed{
   \begin{align}
   P_{\rm cold} &= \kappa \rho^{\Gamma_{\rm cold}}\ ,\\
   \epsilon_{\rm cold} &= \left.\left(\frac{P_{\rm cold}}{\rho}\right)\middle/\left(\Gamma_{\rm cold}-1\right)\right.\ ,\\
   \frac{dP_{\rm cold}}{d\rho} &= \frac{\Gamma_{\rm cold}P_{\rm cold}}{\rho}\ ,\\
   \epsilon_{\rm th} &= \left.\left(\frac{P-P_{\rm cold}}{\rho}\right)\middle/\left(\Gamma_{\rm cold}-1\right)\right.\ ,\\
   h &= 1+\epsilon_{\rm cold}+\epsilon_{\rm th} + \frac{P}{\rho}\ ,\\
   \Gamma_{\rm cold} &= \Gamma_{\rm tab}\ .
   \end{align}}
   $$
   
2. **Case 3: Piecewise Polytrope EOS**

   We now consider a set of dividing densities $\rho_{\min} < \rho_{1} < \cdots < \rho_{n-1} < \rho_{\max}$ such that the pressure and energy density are everywhere continuous. In each interval, the EOS is a single polytrope, with its own $K_{i}$ and $\left(\Gamma_{cold}\right)_{i}\equiv\Gamma_{i}$, so that
   
   $$
    \boxed{
    P_{\rm cold} =
    \left\{
    \begin{matrix}
    K_{0}\rho^{\Gamma_{0}} & , & \rho \leq \rho_{0}\\
    K_{1}\rho^{\Gamma_{1}} & , & \rho_{0} \leq \rho \leq \rho_{1}\\
    \vdots &  & \vdots\\
    K_{j}\rho^{\Gamma_{j}} & , & \rho_{j-1} \leq \rho \leq \rho_{j}\\
    \vdots &  & \vdots\\
    K_{N-1}\rho^{\Gamma_{N-1}} & , & \rho_{N-2} \leq \rho \leq \rho_{N-1}\\
    K_{N}\rho^{\Gamma_{N}} & , & \rho \geq \rho_{N-1}
    \end{matrix}
    \right.
    }\ .
    $$
   
   Then, for each set $\left\{P_{i},K_{i},\Gamma_{i}\right\}$ the desired quantities are computed using the same equations used in Case 2.

In [3]:
%%writefile -a $outfile_path__mhdflux__C


  // First compute P_{cold}, \epsilon_{cold}, dP_{cold}/drho, \epsilon_{th}, h, and \Gamma_{cold},
  // for right and left faces:
  CCTK_REAL P_coldr,eps_coldr,dPcold_drhor=0,eps_thr=0,h_r=0,Gamma_coldr;
  compute_P_cold__eps_cold__dPcold_drho__eps_th__h__Gamma_cold(Ur,eos,Gamma_th,P_coldr,eps_coldr,dPcold_drhor,eps_thr,h_r,Gamma_coldr);
  CCTK_REAL P_coldl,eps_coldl,dPcold_drhol=0,eps_thl=0,h_l=0,Gamma_coldl;
  compute_P_cold__eps_cold__dPcold_drho__eps_th__h__Gamma_cold(Ul,eos,Gamma_th,P_coldl,eps_coldl,dPcold_drhol,eps_thl,h_l,Gamma_coldl);

Appending to ../src/mhdflux.C


<a id='min_max_characteristic_speeds'></a>

# Step 6: The minimum and maximum characteristic speeds \[Back to [top](#toc)\]
$$\label{min_max_characteristic_speeds}$$

In [4]:
%%writefile -a $outfile_path__mhdflux__C

CCTK_REAL smallb4U0_R,smallb4U1_R,smallb4U2_R,smallb4U3_R;
CCTK_REAL smallb4U0_L,smallb4U1_L,smallb4U2_L,smallb4U3_L;
const CCTK_REAL sqrt4pi = sqrt(4.0*M_PI);
#include "NRPy_generated_headers/smallb4R_and_L.h"

  switch(flux_dirn)
  {
      case 1:
      {
#include "NRPy_generated_headers/flux_dirn_x.h"
      }
      case 2:
      {
#include "NRPy_generated_headers/flux_dirn_y.h"
      }
      case 3:
      {
#include "NRPy_generated_headers/flux_dirn_z.h"
      }
  }

}

Appending to ../src/mhdflux.C


<a id='code_validation'></a>

# Step 10: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [5]:
# # Verify if the code generated by this tutorial module
# # matches the original IllinoisGRMHD source code

# # First download the original IllinoisGRMHD source code
# import urllib
# from os import path

# original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/mhdflux.C"
# original_IGM_file_name = "mhdflux-original.C"
# original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# # Then download the original IllinoisGRMHD source code
# # We try it here in a couple of ways in an attempt to keep
# # the code more portable
# try:
#     original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
#     # Write down the file the original IllinoisGRMHD source code
#     with open(original_IGM_file_path,"w") as file:
#         file.write(original_IGM_file_code)
# except:
#     try:
#         original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
#         # Write down the file the original IllinoisGRMHD source code
#         with open(original_IGM_file_path,"w") as file:
#             file.write(original_IGM_file_code)
#     except:
#         # If all else fails, hope wget does the job
#         !wget -O $original_IGM_file_path $original_IGM_file_url

# # Perform validation
# Validation__mhdflux__C  = !diff $original_IGM_file_path $outfile_path__mhdflux__C

# if Validation__mhdflux__C == []:
#     # If the validation passes, we do not need to store the original IGM source code file
#     !rm $original_IGM_file_path
#     print("Validation test for mhdflux.C: PASSED!")
# else:
#     # If the validation fails, we keep the original IGM source code file
#     print("Validation test for mhdflux.C: FAILED!")
#     # We also print out the difference between the code generated
#     # in this tutorial module and the original IGM source code
#     print("Diff:")
#     for diff_line in Validation__mhdflux__C:
#         print(diff_line)

<a id='latex_pdf_output'></a>

# Step 11: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__mhdflux.pdf](Tutorial-IllinoisGRMHD__mhdflux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [6]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path Tutorial-IllinoisGRMHD__mhdflux.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__mhdflux.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__mhdflux.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__mhdflux.tex
!rm -f Tut*.out Tut*.aux Tut*.log